In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from keras.utils.np_utils import to_categorical
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Flatten, Lambda, Activation
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam

train_df = pd.read_json('../input/train.json')
test_df = pd.read_json('../input/test.json')

Using TensorFlow backend.


In [2]:
def std_img(x):
    for i in range(3):
        x[:, :, i] -= np.mean(x[:, :, i].flatten())
        x[:, :, i] /= np.std(x[:, :, i].flatten()) + 1e-7
    return x

def get_image(df):
    '''Create 3-channel 'images'. Return rescale-normalised images.'''
    images = []
    for i, row in df.iterrows():
        # Formulate the bands as 75x75 arrays
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = (band_1 + band_2)/2

        # Rescale
        img = np.dstack([band_1,band_2,band_3])
        img = std_img(img)

        images.append(img)
    return np.array(images)


train_x = get_image(train_df)
test_x = get_image(test_df)

print(train_x.shape,test_x.shape)

(1604, 75, 75, 3) (8424, 75, 75, 3)


In [3]:
y = train_df.is_iceberg.values
print(y[:5])

[0 0 1 0 0]


In [4]:
from keras.callbacks import ModelCheckpoint,LearningRateScheduler
from keras.models import Model
from keras.applications.xception import Xception
def create_model():
    base_model = Xception(include_top=False, weights=None, input_tensor=None, input_shape=(75,75,3), pooling='max')
    x = Dense(512,activation='relu')(base_model.output)
    x = Dropout(0.2)(x)
    x = Dense(1,activation='sigmoid')(x)
    return Model(inputs=base_model.input, outputs=x)
print('model model')


tmp_m = create_model()
tmp_m.summary()

model model
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 75, 75, 3)    0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 37, 37, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 37, 37, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 37, 37, 32)   0           block1_conv1_bn[0][0]            
_________________________________________________________________________________________________

In [5]:
from sklearn.model_selection import KFold
from keras.preprocessing.image import ImageDataGenerator
import gc

def lr_f(epoch):
    if epoch<10:
        return 0.001
    elif epoch<30:
        return 0.0005
    else:
        return 0.0001

def kfold_train(fold_cnt=3,rnd=233):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.05,
            height_shift_range=0.05,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 12
        steps_train = len(curr_y)//bat_size
        
        
        model = create_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=70, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        
        del model
        gc.collect()
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=3)

Epoch 1/70
Epoch 00001: val_loss improved from inf to 0.69241, saving model to best_m.h5
 - 22s - loss: 0.7253 - acc: 0.6891 - val_loss: 0.6924 - val_acc: 0.5533
Epoch 2/70
Epoch 00002: val_loss did not improve
 - 14s - loss: 0.6057 - acc: 0.7257 - val_loss: 0.6968 - val_acc: 0.4467
Epoch 3/70
Epoch 00003: val_loss did not improve
 - 14s - loss: 0.4541 - acc: 0.8080 - val_loss: 0.7114 - val_acc: 0.4467
Epoch 4/70
Epoch 00004: val_loss did not improve
 - 14s - loss: 0.4607 - acc: 0.7922 - val_loss: 0.7220 - val_acc: 0.4467
Epoch 5/70
Epoch 00005: val_loss did not improve
 - 15s - loss: 0.5083 - acc: 0.7754 - val_loss: 0.8782 - val_acc: 0.4467
Epoch 6/70
Epoch 00006: val_loss improved from 0.69241 to 0.50515, saving model to best_m.h5
 - 16s - loss: 0.4615 - acc: 0.8025 - val_loss: 0.5051 - val_acc: 0.8187
Epoch 7/70
Epoch 00007: val_loss improved from 0.50515 to 0.40607, saving model to best_m.h5
 - 16s - loss: 0.4537 - acc: 0.8081 - val_loss: 0.4061 - val_acc: 0.8168
Epoch 8/70
Epoch 0

Epoch 66/70
Epoch 00066: val_loss did not improve
 - 14s - loss: 0.2082 - acc: 0.9251 - val_loss: 0.2502 - val_acc: 0.8972
Epoch 67/70
Epoch 00067: val_loss did not improve
 - 14s - loss: 0.1786 - acc: 0.9242 - val_loss: 0.2552 - val_acc: 0.9009
Epoch 68/70
Epoch 00068: val_loss did not improve
 - 14s - loss: 0.2058 - acc: 0.9129 - val_loss: 0.2425 - val_acc: 0.9084
Epoch 69/70
Epoch 00069: val_loss did not improve
 - 14s - loss: 0.1726 - acc: 0.9196 - val_loss: 0.2843 - val_acc: 0.9065
Epoch 70/70
Epoch 00070: val_loss did not improve
 - 14s - loss: 0.1897 - acc: 0.9204 - val_loss: 0.2751 - val_acc: 0.9065
Epoch 1/70
Epoch 00001: val_loss improved from inf to 0.69322, saving model to best_m.h5
 - 33s - loss: 0.7940 - acc: 0.6282 - val_loss: 0.6932 - val_acc: 0.4860
Epoch 2/70
Epoch 00002: val_loss did not improve
 - 14s - loss: 0.7452 - acc: 0.6761 - val_loss: 0.6951 - val_acc: 0.4860
Epoch 3/70
Epoch 00003: val_loss did not improve
 - 14s - loss: 0.5712 - acc: 0.7584 - val_loss: 0.70

Epoch 57/70
Epoch 00057: val_loss did not improve
 - 14s - loss: 0.2930 - acc: 0.8839 - val_loss: 0.3370 - val_acc: 0.8561
Epoch 58/70
Epoch 00058: val_loss did not improve
 - 14s - loss: 0.2649 - acc: 0.8951 - val_loss: 0.2896 - val_acc: 0.8542
Epoch 59/70
Epoch 00059: val_loss did not improve
 - 14s - loss: 0.3164 - acc: 0.8690 - val_loss: 0.2117 - val_acc: 0.9084
Epoch 60/70
Epoch 00060: val_loss did not improve
 - 14s - loss: 0.2689 - acc: 0.8848 - val_loss: 0.2412 - val_acc: 0.8879
Epoch 61/70
Epoch 00061: val_loss did not improve
 - 14s - loss: 0.2514 - acc: 0.9026 - val_loss: 0.3123 - val_acc: 0.8785
Epoch 62/70
Epoch 00062: val_loss did not improve
 - 14s - loss: 0.2562 - acc: 0.8970 - val_loss: 0.2224 - val_acc: 0.8972
Epoch 63/70
Epoch 00063: val_loss did not improve
 - 14s - loss: 0.2696 - acc: 0.8923 - val_loss: 0.7511 - val_acc: 0.8561
Epoch 64/70
Epoch 00064: val_loss did not improve
 - 14s - loss: 0.2531 - acc: 0.8887 - val_loss: 0.2103 - val_acc: 0.9009
Epoch 65/70
Epoc

 - 16s - loss: 0.2295 - acc: 0.8979 - val_loss: 0.2209 - val_acc: 0.8951
Epoch 50/70
Epoch 00050: val_loss did not improve
 - 15s - loss: 0.2630 - acc: 0.8831 - val_loss: 0.2498 - val_acc: 0.8895
Epoch 51/70
Epoch 00051: val_loss improved from 0.22086 to 0.21190, saving model to best_m.h5
 - 18s - loss: 0.2661 - acc: 0.8933 - val_loss: 0.2119 - val_acc: 0.9064
Epoch 52/70
Epoch 00052: val_loss did not improve
 - 15s - loss: 0.2206 - acc: 0.9026 - val_loss: 0.2220 - val_acc: 0.9064
Epoch 53/70
Epoch 00053: val_loss did not improve
 - 15s - loss: 0.1984 - acc: 0.9148 - val_loss: 0.2364 - val_acc: 0.8933
Epoch 54/70
Epoch 00054: val_loss did not improve
 - 15s - loss: 0.2181 - acc: 0.9101 - val_loss: 0.2656 - val_acc: 0.8801
Epoch 55/70
Epoch 00055: val_loss did not improve
 - 15s - loss: 0.2518 - acc: 0.8961 - val_loss: 0.2201 - val_acc: 0.9120
Epoch 56/70
Epoch 00056: val_loss did not improve
 - 15s - loss: 0.2382 - acc: 0.8971 - val_loss: 0.2228 - val_acc: 0.9157
Epoch 57/70
Epoch 0005

In [6]:
import pickle
with open('../features/xception_aug1_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
from sklearn.metrics import log_loss
print(log_loss(y,train_pred))
    
submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/xception_aug1_sub.csv', index=False)

0.212381384877
         id  is_iceberg
0  5941774d    0.038126
1  4023181e    0.899046
2  b20200e4    0.327146
3  e7f018bb    0.973960
4  4371c8c3    0.401795
